In [6]:
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True)
def normalize(df):
    result = df.copy()
    for column in df.columns:
        max_value = df[column].max()
        min_value = df[column].min()
        if(max_value-min_value!=0):
            result[column] = (df[column] - min_value) / (max_value - min_value) 
        else:
            result[column]=df[column]
    return(result)  
normalised_train=normalize(pd.read_csv("./medical_training_data.csv"))
normalised_test=normalize(pd.read_csv("./medical_testing_data.csv"))


normalised_train_data=np.transpose(normalised_train.iloc[:,0:16].to_numpy())


train_result=normalised_train.iloc[:,16].to_numpy()
train_result=np.transpose(train_result[:,np.newaxis])

normalised_test_data=np.transpose(normalised_test.iloc[:,0:16].to_numpy())

test_result=normalised_test.iloc[:,16].to_numpy()
test_result=np.transpose(test_result[:,np.newaxis])

class ann:
    def __init__(self,train,train_result,test,test_result,epoches,alpha,hidden_layers,l):  # l to input the activation functions between each layer  #  hidden layers is a list passed in which number of features for each hidden layer 
        
        self.train=train
        self.train_result=train_result
        self.test=test 
        self.train_result=test_result
        self.epoches=epoches
        self.alpha=alpha
        self.l=l          # activation function between each rows

        self.input_features=np.shape(train)[0] # along the rows 
        self.output_features = np.shape(train_result)[0]
        self.m=np.shape(train)[1]      # along the columns in a row    

        self.layers=[self.input_features]+hidden_layers+[self.output_features]   # contains number of features in each layer including start , hidden and end layers


        weights=[]     
        bias=[]
         
        
        error_wrt_z=[]
        error_wrt_w=[]
        error_wrt_b=[]

        for i in range(len(self.layers)-1): 
            a=np.random.rand (self.layers[i+1],self.layers[i])
            weights.append(a)

            b=np.random.rand(self.layers[i+1],1)          # will use broad casting                                              
            bias.append(b) 

            error_wrt_w.append(i)    # how many dw and db will be there , later i's will be updated with dw and db matrices 

            error_wrt_b.append(i)      # there will be number of (layers - 1) matrices for weights,bias,error wrt w,b,z in each forward propagation   

            error_wrt_z.append(i)  
        
        self.weights=weights
        self.bias=bias

       
        self.error_wrt_z= error_wrt_z
        self.error_wrt_w= error_wrt_w
        self.error_wrt_b= error_wrt_b

        self.activations=[]
        self.z=[]

        self.function_derivitive=[self.reluDerivative,self.sigmoid_derivative]  # in middle layers put relu or sigmoid only you cannnot put softmax in middle as you dont know d softmax(z) / d(z)
        self.function=[self.relu,self.sigmoid,self.softmax]  

    

    def relu(self,x):
        return np.maximum(0.0, x)


    def sigmoid(self , x):
        y=1/(1+np.exp(-x) )                       
        return y
        
    def sigmoid_derivative(self, x):        #  d sigma(z) / d(z) = sigma(z)*(1-sigma(z)) 
        return np.multiply(x,(1-x))         # in this we are finding for all the a's of the matrix 

    def reluDerivative(self,x):
        y=np.copy(x)
        y[y<=0] = 0
        y[y>0] = 1
        return y     

    def softmax (self,x):
        e=np.exp(x)
        return e/(np.sum(e,axis=0))  


    def forward_propogate(self):
        self.activations.clear()   #  with each epoch our w will be updated so we will be getting new activations as well as z  
        self.z.clear()
        self.z.append(self.train)
        self.activations.append(self.train)        ### note z and a will contain all the layers from  1st given layer at indeex 0 to the last output predicted layer 
        for i in range (len(self.layers)-1):
            self.z.append(np.add (np.dot(self.weights[i],self.activations[i]) ,self.bias[i]))
        
           # self.z.append(np.add( np.dot( self.weights[i],self.activations[i] ) ,np.repeat(self.bias[i],self.m,axis=1)) )    
            self.activations.append(self.function[self.l[i]](self.z[i+1]  ) )     

            
a=ann(train=normalised_train_data,train_result=train_result,test=normalised_test_data,test_result=test_result,epoches=1,alpha=0.01,hidden_layers=[3],l=[1,2])
            


a.forward_propogate()
for i in a.z:
    print(i.shape)

for i in a.activations:
    print(i.shape)

        

(16, 2915)
(3, 2915)
(1, 2915)
